In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from bing_helper import load_json_files, get_hits_from_json, load_lang_news_hits
from regression import fit_regression_and_rename_coeffs_by_cat, fit_regression_and_rename_coeffs, write_reg_results
from processing import filter_by_region_cat_code, build_regression_dataframe, load_preprocessed_events
from plot_regression import plot_regression_results, plot_regression_results_interactions, plot_country_counts, plot_regression_results_interactions_from_dict, plot_cat_by_cat_variable, BASELINE_DICT, CAT_DICT, LABEL_RENAME_DICT, LABEL_SORT_DICT
from preprocessing import load_if_in_country_or_lang
from regression import transform_vars_for_regression, fit_logit_and_rename_coeffs, fit_regression_and_rename_coeffs_by_cat, nagelkerke, mcfadden, likelihood_ratio_test
import statsmodels.api as sm
import statsmodels.formula.api as smf
from processing import calculate_uniqueness, calculate_uniqueness_two_columns

In [5]:
df_crawled = pd.read_csv('events/new/processed_0_manually_with_wikiviews.csv').drop_duplicates()

In [6]:
df_crawled.event_date = pd.to_datetime(df_crawled.event_date)
df_crawled = calculate_uniqueness(df_crawled, 30, 'country')
df_crawled = calculate_uniqueness(df_crawled, 30, 'cat')
df_crawled = calculate_uniqueness_two_columns(df_crawled, 30, 'country', 'cat')
df_reg = df_crawled[df_crawled.year > 2015].copy()

In [7]:
df_reg['noticed'] = df_reg.views_7_sum > 10
df_reg['noticed_int'] = df_reg.noticed.astype(int)
df_reg = transform_vars_for_regression(df_reg)

In [8]:
df_reg['views_7_max'] = df_reg.list_views_7_days.apply(lambda s: max([int(i) for i in s[1:-1].split(' ')]) if len(s) > 2 else 0)

In [39]:
df_reg = df_reg[(df_reg.cat != 'undefined') & (df_reg.event_date.dt.year >= 2016) & (df_reg.views_7_max > 10)].copy()

# Regression

In [10]:
code_reg = 'C(code, Treatment(reference="en"))'
cat_reg = 'C(cat, Treatment(reference="sports"))'
class_reg = 'C(gni_class, Treatment(reference="H"))'
region_reg = 'C(gni_region, Treatment(reference="North America"))'
continent_reg = 'C(continent, Treatment(reference="North America"))'
relevance_reg = 'views_baseline_log'
magnitude = 'population_log'
prominence = 'view_country_article_log'
power_elite = f'GDP_pc_z + {class_reg} + economic_region + oecd'
uniqueness = 'country_articles_log + cat_articles_log + country_cat_articles_log'
independent = f'{cat_reg} + {relevance_reg} + {region_reg} + {magnitude} + {prominence} + {uniqueness} + {power_elite}'


## All News Value features

In [11]:
from regression import fit_nb_with_estimated_alpha, estimate_alpha, fit_nb, standardize_var, fit_nb_with_estimated_alpha_all_codes, standardize_var,transform_vars_for_regression

In [12]:
reg_0 = fit_nb_with_estimated_alpha(
    df_reg,  f'views_7_sum ~ {code_reg} * ({independent})')

Estimated alpha as 15.20 (p=0.0000)
Deviance: 3159.77 | Null-deviance: 4091.57 | Pseudo R² (1 - D/D_0) = 0.2277
Pseudo ChiSq: 13829.82 | Good-Fit-ChiSq: 12923.88 | Residual DF: 12661
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 1.09


In [13]:
reg_1 = fit_nb_with_estimated_alpha(
    df_reg,  f'views_7_max ~ {code_reg} * ({independent})')

Estimated alpha as 166.96 (p=0.0003)
Deviance: 534.15 | Null-deviance: 622.36 | Pseudo R² (1 - D/D_0) = 0.1417
Pseudo ChiSq: 1324.19 | Good-Fit-ChiSq: 12923.88 | Residual DF: 12661
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 0.10


# Select news values

In [20]:
relevance_reg = 'views_baseline_log'
magnitude = 'population_log'
prominence = 'view_country_article_log'
power_elite = f'{class_reg}' #f'GDP_pc_z + {class_reg} + economic_region + oecd'
uniqueness = 'country_articles_log + cat_articles_log'
independent1 = f'{cat_reg} + {relevance_reg} + {region_reg} + {magnitude} + {prominence} + {uniqueness} + {power_elite}'

reg_1b = fit_nb_with_estimated_alpha(df_reg,  f'views_7_max ~ {code_reg} * ({independent1})')
reg_1b.summary2()

Estimated alpha as 169.59 (p=0.0001)
Deviance: 529.53 | Null-deviance: 613.04 | Pseudo R² (1 - D/D_0) = 0.1362
Pseudo ChiSq: 1312.55 | Good-Fit-ChiSq: 12940.04 | Residual DF: 12677
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 0.10


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                 Results: Generalized linear model
====================================================================================================
Model:                           GLM                         AIC:                       259168.7563 
Link Function:                   log                         BIC:                       -119308.7850
Dependent Variable:              views_7_max                 Log-Likelihood:            -1.2951e+05 
Date:                            2022-05-16 12:54            LL-Null:                   -1.2955e+05 
No. Observations:                12749                       Deviance:                  529.53      
Df Model:                        71                          Pearson chi2:              1.31e+03    
Df Residuals:                    12677                       Scale:                     1.0000      
Method:                          IRLS                                                               
----------------------------------------------------------------------------------------------------
                                                     Coef.  Std.Err.    z    P>|z|   [0.025   0.975]
----------------------------------------------------------------------------------------------------
Intercept                                            2.3303   4.3880  0.5310 0.5954  -6.2701 10.9306
code[T.de]                                           2.3244   7.4181  0.3133 0.7540 -12.2148 16.8637
code[T.es]                                          -0.3773   6.7326 -0.0560 0.9553 -13.5729 12.8184
code[T.it]                                           0.9116   7.9562  0.1146 0.9088 -14.6824 16.5055
cat[T.culture]                                       0.4897   0.8103  0.6043 0.5456  -1.0985  2.0778
cat[T.disaster]                                      0.4483   0.7446  0.6020 0.5472  -1.0112  1.9077
cat[T.politics]                                     -0.2668   0.5139 -0.5192 0.6036  -1.2741  0.7404
gni_region[T.East Asia & Pacific]                   -0.0005   0.7501 -0.0006 0.9995  -1.4706  1.4696
gni_region[T.Europe & Central Asia]                 -0.1574   0.6619 -0.2378 0.8121  -1.4546  1.1399
gni_region[T.Latin America & Caribbean]              0.1993   0.9561  0.2084 0.8349  -1.6747  2.0733
gni_region[T.Middle East & North Africa]             0.4709   1.1603  0.4059 0.6848  -1.8031  2.7450
gni_region[T.South Asia]                            -0.3504   1.2557 -0.2790 0.7802  -2.8114  2.1107
gni_region[T.Sub-Saharan Africa]                     0.3545   1.4863  0.2385 0.8115  -2.5586  3.2676
gni_class[T.L]                                       0.7022   1.6193  0.4337 0.6645  -2.4716  3.8761
gni_class[T.LM]                                      0.4192   0.8697  0.4820 0.6298  -1.2854  2.1238
gni_class[T.UM]                                      0.4450   0.6421  0.6930 0.4883  -0.8135  1.7035
code[T.de]:cat[T.culture]                            1.2470   2.2616  0.5514 0.5814  -3.1856  5.6797
code[T.es]:cat[T.culture]                           -0.2294   1.5654 -0.1466 0.8835  -3.2976  2.8387
code[T.it]:cat[T.culture]                            1.1176   2.0854  0.5359 0.5920  -2.9697  5.2049
code[T.de]:cat[T.disaster]                           0.8701   1.8085  0.4811 0.6304  -2.6744  4.4147
code[T.es]:cat[T.disaster]                          -0.2231   1.3220 -0.1688 0.8660  -2.8143  2.3680
code[T.it]:cat[T.disaster]                          -1.4445   2.2542 -0.6408 0.5217  -5.8625  2.9736
code[T.de]:cat[T.politics]                           2.1146   1.4660  1.4424 0.1492  -0.7587  4.9878
code[T.es]:cat[T.politics]                           1.4699   0.9528  1.5427 0.1229  -0.3976  3.3375
code[T.it]:cat[T.politics]                           1.2870   1.6438  0.7829 0.4337  -1.9348  4.5087
code[T.de]:gni_region[T.East Asia & Pacific]         0.0091   1.5918  0.0057 0.9955  -3.1108  3.1289
code[T.es]:gni_region[T.East Asia & Pacific]  

In [21]:
fit1_dict = fit_nb_with_estimated_alpha_all_codes(['en', 'es', 'de', 'it'], df_reg, f'views_7_sum ~ {independent1}', 
    est_method='lbfgs', output_lvl=2)

________________________________________________________________________________________________
================= Fitting en =================
Estimated alpha as 15.94 (p=0.0001)
Deviance: 1632.63 | Null-deviance: 1776.15 | Pseudo R² (1 - D/D_0) = 0.0808
Pseudo ChiSq: 6362.81 | Good-Fit-ChiSq: 6868.21 | Residual DF: 6677
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 0.95
--------------- Summary for en ---------------
                 Generalized Linear Model Regression Results                  
Dep. Variable:            views_7_sum   No. Observations:                 6695
Model:                            GLM   Df Residuals:                     6677
Model Family:        NegativeBinomial   Df Model:                           17
Link Function:                    log   Scale:                          1.0000
Method:                         lbfgs   Log-Likelihood:                -77604.
Date:                Mon, 16 May 2022   Deviance:                       1632.

In [40]:
fit1b_dict = fit_nb_with_estimated_alpha_all_codes(['en', 'es', 'de', 'it'], df_reg, f'views_7_max ~ {independent1}', 
    est_method='lbfgs', output_lvl=2)

________________________________________________________________________________________________
================= Fitting en =================
Estimated alpha as 152.99 (p=0.0000)
Deviance: 143.01 | Null-deviance: 158.68 | Pseudo R² (1 - D/D_0) = 0.0988
Pseudo ChiSq: 477.43 | Good-Fit-ChiSq: 5568.00 | Residual DF: 5396
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 0.09
--------------- Summary for en ---------------
                 Generalized Linear Model Regression Results                  
Dep. Variable:            views_7_max   No. Observations:                 5414
Model:                            GLM   Df Residuals:                     5396
Model Family:        NegativeBinomial   Df Model:                           17
Link Function:                    log   Scale:                          1.0000
Method:                         lbfgs   Log-Likelihood:                -67064.
Date:                Mon, 16 May 2022   Deviance:                       143.01
